# Lecture de l'alphabet des signes sur un flux vidéo

<hr>

In [3]:
#import skimage
import numpy as np
import cv2
import mediapipe as mp
from string import ascii_lowercase
import os
import joblib

import matplotlib.pyplot as plt

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [10]:
LABELS = {k: v for k, v in enumerate(ascii_lowercase)}

#### Création de la fonction de détection et classification des mains/signes sur une image

In [5]:
# création d'unne fonction pour dessiner un rectangle autour de la main détectée
def find_rectangle(hand_landmarks, image):
    image_height, image_width, _ = image.shape
    keypoints = []
    for data_point in hand_landmarks.landmark:
        keypoints.append({
                            'X': data_point.x*image_width,
                            'Y': data_point.y*image_height,                            
                            })
    
    X_min = image_width
    X_max = 0
    Y_min = image_height
    Y_max = 0

    #print(keypoints[0]['X'])
    for x in range(len(keypoints)):
        if (keypoints[x]['X']) < X_min:
            X_min = (keypoints[x]['X'])
        if (keypoints[x]['X']) > X_max:
            X_max = (keypoints[x]['X'])

    for y in range(len(keypoints)):
        if (keypoints[y]['Y']) < Y_min:
            Y_min = (keypoints[y]['Y'])
        if (keypoints[y]['Y']) > Y_max:
            Y_max = (keypoints[y]['Y'])

    startX = int(X_min -0.05* image_width)-10
    startY = int(Y_min -0.05* image_height)-10
    endX = int(X_max +0.05* image_width)+10
    endY = int(Y_max +0.05* image_height)+10
    # rect = cv2.rectangle(image, (startX, startY), (endX, endY), (155, 255, 0), 2)
    return startX, startY, endX, endY

In [6]:
def detect_and_classify_handsign(image):
    
    # imports mediapipe
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    
    with mp_hands.Hands(max_num_hands=1,
                        min_detection_confidence=0.5,
                        min_tracking_confidence=0.5) as hands:
        
        
        # prétraitement de l'image :
        results = hands.process(image)
        
        # on extrait ensuite l'image de la main à partir de l'image d'origine et des coordonnées 
        # du rectangle d'encadrement :
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                ## récupération des coordonnées des 4 coins du rectangle grace à la fonction find_rectangle.
                x1, y1, x2, y2 = find_rectangle(hand_landmarks, image)
                ## extraction de l'image de la main et prétraîtement en vue du traîtement par le modèle de 
                ## classification (rognage, conversion en niveaux de gris, standartisation, modification de forme).
                cropped = image[y1:y2, x1:x2]
                gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY) 
                localized = (cv2.resize(gray,(28,28)).astype('float') / 255).reshape(1,784)
                
                # reconaissance du signe grace au modèle de classification :
                recognized = np.argmax(model_rec.predict(localized)) 
                print(LABELS[recognized])
                # on trace le rectangle et affiche la lettre reconnue
                image = cv2.rectangle(image, (x1, y1), (x2, y2), (122,10,122), 2) 
                image = cv2.putText(image, LABELS[recognized], (10,300), cv2.FONT_HERSHEY_SIMPLEX, 2, (122,10,122), 2)
    
    return image

#### Chargement de modèle de classification

In [7]:
# modèle de classification du signe, entraîné avec Keras
model_rec = tf.keras.models.load_model('../models/detection_langue_des_signes2.h5')
#model_rec = joblib.load('../models/classification_lsf.sav')

#### Création de la fonction de capture et de traitement de l'image Webcam

In [8]:
def process_video(record_name=None):
    cap = cv2.VideoCapture(0)

    if (cap.isOpened()== False): 
        print("Error opening video stream")
    else :
        # on récupère la hauteur et la largeur de l'image grâce aux méthodes cv2
        global_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        global_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if record_name: # enregistement si un nom de fichier a été spécifié
        result = cv2.VideoWriter(record_name,  cv2.VideoWriter_fourcc(*'MJPG'), 15, (global_width, global_height)) 
        
    while(cap.isOpened()):
        ret, frame = cap.read()
        maximum_fps = 10
        counter = 0
        
        if ret == True:
            if (counter % maximum_fps) == 0: ## Une image sur cinq
                # on appelle la fonction créér ci-dessus pour traîter l'image video
                frame_processed = detect_and_classify_handsign(frame)
                # on retourne l'image retraitée
                cv2.imshow('Frame',frame_processed)
            counter += 1

            if cv2.waitKey(1) == 27:
                break

        else: 
            break

    cap.release()
    if record_name:
        result.release()
    cv2.destroyAllWindows()

In [ ]:
process_video()